# Intro to Data Science @ SzISz Part VII.
## Grand Finale:<br> Model Selection, Pipeline building, and Custom Sklearn Nodes 

### Table of contents
- <a href="#What-is-Model-Selection?">Model Selection Theory</a>
- <a href="#Cross-Validation">Cross Validation</a>
    - <a href="#Grid-Search-Cross-Validation">Randomized Search Cross Validation</a>
    - <a href="#Randomized-Search-Cross-Validation">Randomized Search Cross Validation</a>
- <a href="#Building-Pipelines">Building Pipelines</a>
- <a href="#Building-a-Custom-Sklearn-Node">Building a Custom Scikit-learn Node</a>
    

## What is Model Selection?
_"Model selection is the task of selecting a statistical model from a set of candidate models, given data. In the simplest cases, a pre-existing set of data is considered."_ from: <a href="https://en.wikipedia.org/wiki/Model_selection">Wiki</a>  
In this context we also include the process of finding the optimal hyperparameters.

## Why is it important?
To find the optimal solution to a given problem, one must train several models with similar predictive/exploratory power and select the simplest one. This process includes selecting models and finding optimal hyperparameters which is a time consuming and tedious work when done by hand. We use automatized solutions to overcome this problem, save time, and yield better results.

## Tools
- Grid Search
- Randomized search
- etc.

In [ ]:
%matplotlib inline
import numpy as np
import scipy.sparse as sp
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_digits, fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import RandomizedPCA, TruncatedSVD
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC

np.random.seed = 42

In [ ]:
digits = load_digits()
X_dig, y_dig = digits.data, digits.target

In [ ]:
categories = [
    'alt.atheism',
    'talk.religion.misc',
]
twentynews = fetch_20newsgroups(subset='train', categories=categories)
X_20, y_20 = twentynews.data, twentynews.target

## <a href="http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation">Cross Validation</a>

In order to select an optimal model, first one must be able to measure a model's/pipe's accuracy.  

First, one must select a valid metric for the model. In sklearn, the basic validation metric is accuracy score in case of classification, and $r^{2}$ for regression. Altough, several other metrics can be selected from <a href="http://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules">this</a> list.

_"Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called overfitting."_ [1]. To overcome this problem, one must split the data to __training__ and __test__ dataset; train the model on the train dataset, then measure the precision on the test dataset.

However different splits can produce different outcomes, so this process must be repeated several times to give a good approximation to the examined model's accuracy. This process is called __Cross Validation__ and there are <a href="http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators">different strategies</a> to make these splits.

A simple model can yield different solutions to the same data based on its hyperparameters so multiple models must be trained to select the ideal hyperparamter settings. Cross Validation gives a good approximation to a trained model's accuracy, but additional methods are required to select the ideal hyperparameters. 

[1] <a href="http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-evaluating-estimator-performance">Scikit-learn 0.17.1 documentation</a>


### Grid Search Cross Validation
Grid search is a method which generates a parameter grid from a list of settings, and measure the input model's accuracy in every setting using cross validation.

In [ ]:
from sklearn.grid_search import GridSearchCV

---
## Intermission:
## Building Pipelines
A Quick reminder how to build sklearn pipes and how to access their parameters.

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion

In [ ]:
steps = [
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier())
]
pipe_20 = Pipeline(steps=steps)

In [ ]:
pipe_20.steps

In [ ]:
pipe_20.get_params()

In [ ]:
pipe_20.set_params(tfidf__norm='l1')

In [ ]:
pipe_20.get_params()

__end of intermission__

---

In [ ]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

In [ ]:
pipe_digit = Pipeline(steps=[('pca', RandomizedPCA()), ('logistic', LogisticRegression())])

In [ ]:
param_grid = [
    {'pca__n_components': [20, 40, 64], 'logistic__C': np.logspace(-4, 4, 3)},
]

In [ ]:
grid_search = GridSearchCV(estimator=pipe_digit, param_grid=param_grid, verbose=1)
grid_search

In [ ]:
grid_search.fit(X_dig, y_dig)

In [ ]:
grid_search.best_estimator_.get_params(deep=False)

In [ ]:
grid_search.best_params_, grid_search.best_score_

### Randomized Search Cross Validation
Randomized search randomly generate a fixed number of hyperparameter setups. It selects the parameters from the provided parameter parameter ranges and then measures them with cross validation.

In [ ]:
from sklearn.grid_search import RandomizedSearchCV

In [ ]:
param_dist = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__n_iter': (10, 50, 80),
}
n_iter_search = 30

In [ ]:
random_search = RandomizedSearchCV(estimator=pipe, param_distributions=param_dist, verbose=2 ,random_state=42)
random_search

In [ ]:
random_search.fit(X_20, y_20)

In [ ]:
random_search.best_estimator_.get_params(deep=False)

In [ ]:
random_search.best_params_, random_search.best_score_

## Building a Custom Sklearn Node

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, ClusterMixin

In [ ]:
class MyEstimator(BaseEstimator, TransformerMixin):
    pass

In [ ]:
class MyClassifier(BaseEstimator, ClassifierMixin):
    pass

In [ ]:
class MyClustering(BaseEstimator, ClusterMixin):
    pass